# Binary search trees


## Definition

A binary search tree is a tree where
- each node stores a value
- each node is either a leaf or has one or two children
- the left child has a value smaller than the node, and the right larger

![bst](resources/bst.png)
Topics to worry about

- Finding an element
- Inserting and element
- Reversing the tree
- Creating the tree

In [1]:
import unittest
from random import randint

In [2]:
class Node:
    """
    Node class
    """
    def __init__(self, value):
        self.value = value
        self.children = [None, None]

    def get_left(self):
        return self.children[0]
    
    def set_left(self, node):
        self.children[0] = node

    def get_right(self):
        return self.children[1]
    
    def set_right(self, node):
        self.children[1] = node
    
    def set_to(self, other):
        self = other
    
    def __repr__(self):
        r = "none" if self.children[1] is None else self.children[1].value
        l = "none" if self.children[0] is None else self.children[0].value
        return f"(key = {self.value}, ({l}, {r}))"
        

In [3]:
four = Node(4)
four.set_left(Node(5))
four

(key = 4, (5, none))

## Complexity

Search and insertion are proportional to he height of the tree. In a worst case scenario the tree is a continuous line, so the complexity is $O(n)$.

## Search

The key insight is that moving on each subnode of a given node gives takes you to the top of another binary search tree (subtree if you will). This directly means that one could write a recursive solution for finding a given key.

In the worst  case scenario the binary search tree works like a linked list, so searching through it is an $O(n)$ operation. Given a balanced  search tree, the search is run in $O(h)$, where $h ~ log(n)$. Observe similarities to binary search on a sorted array.

## Insertion

Query the current root. If it's equal to the key we want inserted, ignore (WARNING: assumption). If it's smaller than the key we want inserted, we query the children nodes.

## In order traversal

Recursively traverse the left side and the right side of the binary search tree. Notice similarities to [Merge Sort](https://en.wikipedia.org/wiki/Merge_sort#Top-down_implementation_using_lists) in the returns. 

One needs to propagate all the values from the recursively received values up, and `yield` has the advantage of being both more memory efficient and requiring more clean code than propagating lists upwards and trying to unpack and repack them. 

In [4]:
class BST:
    def __init__(self):
        # Head node
        self.leader = None

    def insert(self, key):
        # initialise if needed
        if self.leader == None:
            self.leader = Node(key)
        else:
            self.insert_recursive(self.leader, key)

    def insert_recursive(self, root, key):
        # ignoring insertion if key already exists
        if root.value > key:
            # go left 
            if root.get_left() is None:
                # if right child doesn't  exist, put node
                root.set_left(Node(key))
            else:
                self.insert_recursive(root.get_left(), key)
            
        elif root.value < key:
            # go right
            if root.get_right() is None:
                # if right child doesn't  exist, put node
                root.set_right(Node(key))
            else:
                self.insert_recursive(root.get_right(), key)

    def find(self, key):
        return self._find_recursive(self.leader, key)
    
    def _find_recursive(self, root, key):
        if root is None:
            # Didn't find the key
            return False # Maybe raise KeyError
        elif root.value == key:
            # Found the key
            return True
        elif root.value > key:
            # go left
            return self._find_recursive(root.get_left(), key)
        elif root.value < key:
            # go right
            return self._find_recursive(root.get_right(), key)
        
    def inorder_traversal(self):
        return self._inorder_recursive(self.leader)
    
    def _inorder_recursive(self, root):
        if root is not None:
            if root.get_left() is not None:
                for left_node in self._inorder_recursive(root.get_left()):
                    yield left_node
            
            yield root.value
            
            if root.get_right() is not None:
                for right_node in self._inorder_recursive(root.get_right()):
                    yield right_node
    
    def max_depth(self):
        return self._max_depth_recursive(self.leader)
    
    def _max_depth_recursive(self, root):
        if root is None:
            return 0
        else:
            return 1 + max(self._max_depth_recursive(root.get_left() ), 
                           self._max_depth_recursive(root.get_right()))
    
    def delete_key(self, key):
        self._delete_leaf(self.leader, key)
        
    def _delete_leaf(self, root, key):
        print(f'{root} {key}')
        if root is None:
            pass # Again, maybe raise KeyError
        elif root.value == key:
            left_child = root.get_left()
            right_child = root.get_right()
            if (left_child is None) and (right_child is None):
                print('Noning')
                # set root to None
                del root
            elif (left_child is None):
                # set root to right child
                root.set_to(right_child)
            elif (right_child is None):
                # set root to left_child
                root.set_to(left_child)
            elif (left_child is not None) and (right_child is not None):
                print(f'replacing {root} with {right_child}')
                root.value = right_child.value
                root.set_left(left_child)
                root.set_right(right_child)
                new_key = right_child.value
                print(f'new root {root}')
                self._delete_leaf(right_child, new_key)
        elif root.value > key:
            # go left
            self._delete_leaf(root.get_left(), key)
        elif root.value < key:
            # go right
            self._delete_leaf(root.get_right(), key)
            

In [5]:
rand_keys = [randint(0, 20) for _ in range(100)]
bst = BST()

for k in rand_keys:
    bst.insert(k)
head = bst.leader.value
found = [bst.find(x) for x in range(0, 20)]
print(list(bst.inorder_traversal()))

class TestOrder(unittest.TestCase):
    def runTest(self):
        # won't always pass as there's the odd chance that some number won't be inserted
        self.assertListEqual(list(bst.inorder_traversal()),
                              list(range(21)))
        print('Test passed')

TestOrder().runTest()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Test passed


## Deletion

Take the example binary search tree
![img](resources/bst.png)

Removing a key could lead to the following cases:

1. **key is a leaf** -- simply remove and continue
2. **key has one child node** -- replace node with child node and proceed
3. **key has two children nodes** -- find in order successor and replace contents of the key node with the in order successor. Replace in order successor with it's in order succesor, etc.

Complexity deletion $O(h)$.

In [8]:
bst2 = BST()
bst2.insert(8)
bst2.insert(3)
bst2.insert(1)
bst2.insert(6)
bst2.insert(4)
bst2.insert(7)
bst2.insert(10)
bst2.insert(14)
bst2.insert(13)
print(list(bst2.inorder_traversal()))
# Delete leaf
bst2.delete_key(3)
print(list(bst2.inorder_traversal()))

[1, 3, 4, 6, 7, 8, 10, 13, 14]
(key = 8, (3, 10)) 3
(key = 3, (1, 6)) 3
replacing (key = 3, (1, 6)) with (key = 6, (4, 7))
new root (key = 6, (1, 6))
(key = 6, (4, 7)) 6
replacing (key = 6, (4, 7)) with (key = 7, (none, none))
new root (key = 7, (4, 7))
(key = 7, (none, none)) 7
Noning
[1, 6, 4, 7, 7, 8, 10, 13, 14]


# Advantages of BST

- Unlike hash maps, binary search trees have linear insertion and deletion times.
- Inorder traversal is $O(n)$ too, which is way better than any sorting obtainable from hash maps.
- Hash table resizing is quite costly, can be avoided when working with binary search trees

## Use cases

Huffman codes, heaps, heap sort, hashing in p2p networks.


# Further reading

[Geeks for Geeks](http://www.geeksforgeeks.org/binary-search-tree-set-1-search-and-insertion/)

[Basic Python Implementation Gist](https://gist.github.com/jakemmarsh/8273963)

[Moar implementations in Python](http://interactivepython.org/runestone/static/pythonds/Trees/SearchTreeImplementation.html)